In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\hu041\OneDrive\桌面\nschool\Spotify_Youtube.csv")  #載入CSV檔

In [3]:
print(df.head(5)) #顯示前五筆

   Unnamed: 0    Artist                                        Url_spotify  \
0           0  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
1           1  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
2           2  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
3           3  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
4           4  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   

                                           Track  \
0                                 Feel Good Inc.   
1                                Rhinestone Eyes   
2  New Gold (feat. Tame Impala and Bootie Brown)   
3                             On Melancholy Hill   
4                                 Clint Eastwood   

                                           Album Album_type  \
0                                     Demon Days      album   
1                                  Plastic Beach      album   
2  New Gold (feat. Tame Impal

In [4]:
print(df.isna().sum())  #檢查遺漏值

Unnamed: 0            0
Artist                0
Url_spotify           0
Track                 0
Album                 0
Album_type            0
Uri                   0
Danceability          2
Energy                2
Key                   2
Loudness              2
Speechiness           2
Acousticness          2
Instrumentalness      2
Liveness              2
Valence               2
Tempo                 2
Duration_ms           2
Url_youtube         470
Title               470
Channel             470
Views               470
Likes               541
Comments            569
Description         876
Licensed            470
official_video      470
Stream              576
dtype: int64


In [5]:
print(df.describe())  #快速統計

         Unnamed: 0  Danceability        Energy           Key      Loudness  \
count  20718.000000  20716.000000  20716.000000  20716.000000  20716.000000   
mean   10358.500000      0.619777      0.635250      5.300348     -7.671680   
std     5980.915774      0.165272      0.214147      3.576449      4.632749   
min        0.000000      0.000000      0.000020      0.000000    -46.251000   
25%     5179.250000      0.518000      0.507000      2.000000     -8.858000   
50%    10358.500000      0.637000      0.666000      5.000000     -6.536000   
75%    15537.750000      0.740250      0.798000      8.000000     -4.931000   
max    20717.000000      0.975000      1.000000     11.000000      0.920000   

        Speechiness  Acousticness  Instrumentalness      Liveness  \
count  20716.000000  20716.000000      20716.000000  20716.000000   
mean       0.096456      0.291535          0.055962      0.193521   
std        0.111960      0.286299          0.193262      0.168531   
min        0

In [6]:
def _summarize_missing(df, max_rows=20):
    na = df.isna().sum()
    na = na[na > 0].sort_values(ascending=False)
    if na.empty:
        print(" 無缺失值。")
    else:
        print(" 缺失值統計（>0 的欄位）：")
        to_show = na.head(max_rows)
        print(to_show.to_string())
        if len(na) > max_rows:
            print(f"  ... 另有 {len(na) - max_rows} 個欄位省略")

In [7]:
def clean_data_impute(df: pd.DataFrame) -> pd.DataFrame:
    print("===== 原始資料摘要 =====")
    print(f"shape: {df.shape}")
    _summarize_missing(df)
    print("\n===== 刪除冗餘欄位 =====")
    drop_cols = [c for c in ["Unnamed: 0"] if c in df.columns]
    if drop_cols:
        print(f"  將移除欄位：{drop_cols}")
        df = df.drop(columns=drop_cols, errors="ignore")
    else:
        print("  無可移除的冗餘欄。")
    print(f"  -> shape: {df.shape}")
    print("\n=====型別標準化（數值欄轉 numeric）=====")
    numeric_candidates = [
        "Danceability","Energy","Key","Loudness","Speechiness","Acousticness",
        "Instrumentalness","Liveness","Valence","Tempo","Duration_ms",
        "Views","Likes","Comments","Stream"
    ]
    exist_numeric = [c for c in numeric_candidates if c in df.columns]
    print(f"  嘗試轉為數值的欄位：{exist_numeric}")
    before_na = df[exist_numeric].isna().sum()
    for c in exist_numeric:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    after_na = df[exist_numeric].isna().sum()
    diff = (after_na - before_na)
    if not diff.empty:
        print("  轉型造成的 NaN 變化（正值=新增 NaN，代表原本是非數字字串）：")
        print(diff[diff != 0].to_string())
    print(f"  -> shape: {df.shape}")
    print("\n===== 布林欄位正規化 =====")
    bool_cols = [c for c in ["Licensed", "official_video"] if c in df.columns]
    if bool_cols:
        print(f"  嘗試正規化布林欄位：{bool_cols}")
        for c in bool_cols:
            # 先統一字串，保留 NaN
            df[c] = df[c].apply(lambda x: str(x).strip().lower() if pd.notna(x) else np.nan)
            df[c] = df[c].map({"true": True, "false": False, "1": True, "0": False})
        for c in bool_cols:
            vc = df[c].value_counts(dropna=False)
            print(f"  {c} 值分佈：\n{vc.to_string()}")
    else:
        print("  無布林欄位可處理。")
    print(f"  -> shape: {df.shape}")
    print("\n===== 數值缺失填補（中位數）=====")
    filled_counts = {}
    for c in exist_numeric:
        n_before = df[c].isna().sum()
        if n_before > 0:
            median_val = df[c].median(skipna=True)
            df[c] = df[c].fillna(median_val)
            n_after = df[c].isna().sum()
            filled_counts[c] = n_before - n_after
    if filled_counts:
        print("  各欄填補的缺失數量：")
        print(pd.Series(filled_counts).sort_values(ascending=False).to_string())
    else:
        print("  數值欄無需填補。")
    _summarize_missing(df)  # 檢查剩餘缺失
    print(f"  -> shape: {df.shape}")
    print("\n===== 布林缺失填補（眾數）=====")
    filled_bool = {}

    for c in bool_cols:
        # 確保是可含 NA 的布林 dtype（避免 downcasting 警告）
        df[c] = df[c].astype("boolean")

        n_before = df[c].isna().sum()
        if n_before > 0:
            # 以眾數填補；若沒眾數（全是 NA），就用 False
            mode_val = df[c].mode(dropna=True)
            fill_val = bool(mode_val.iloc[0]) if not mode_val.empty else False

            # 不用 inplace，避免 chained assignment 問題
            df.loc[:, c] = df[c].fillna(fill_val)

            n_after = df[c].isna().sum()
            filled_bool[c] = n_before - n_after

        if filled_bool:
            print("  各布林欄填補的缺失數量：")
            print(pd.Series(filled_bool).sort_values(ascending=False).to_string())
        else:
            print("  布林欄無需填補。")

    _summarize_missing(df)
    print(f"  -> shape: {df.shape}")
    print("\n===== 文字欄位缺失填補（空字串）=====")
    text_candidates = [
        "Artist","Track","Album","Album_type","Uri","Title","Channel","Description",
        "Url_spotify","Url_youtube"
    ]
    text_cols = [c for c in text_candidates if c in df.columns]
    filled_text = {}
    for c in text_cols:
        n_before = df[c].isna().sum()
        if n_before > 0:
            df[text_cols] = df[text_cols].fillna("")
            n_after = df[c].isna().sum()
            filled_text[c] = n_before - n_after
    if filled_text:
        print("  各文字欄填補的缺失數量：")
        print(pd.Series(filled_text).sort_values(ascending=False).to_string())
    else:
        print("  文字欄無需填補。")
    _summarize_missing(df)
    print(f"  -> shape: {df.shape}")
    print("\n===== 收尾處理（殘餘缺失）=====")
    remaining_na = df.isna().sum()
    remaining_na = remaining_na[remaining_na > 0]
    if not remaining_na.empty:
        print("  尚有缺失的欄位：")
        print(remaining_na.to_string())
        for c in remaining_na.index:
            if c in exist_numeric:
                df[c].fillna(df[c].median(skipna=True), inplace=True)
            elif c in bool_cols:
                df[c].fillna(False, inplace=True)
            else:
                df[c].fillna("", inplace=True)
        print("  -> 已依欄型別做最保守填補。")
    else:
        print("  ✅ 無殘餘缺失。")
    _summarize_missing(df)
    print(f"  -> shape: {df.shape}")
    print("\n===== 移除完全重複列（保留第一筆）=====")
    before = len(df)
    df = df.drop_duplicates(keep="first")
    dropped = before - len(df)
    print(f"  -> 移除 {dropped} 列完全重複資料。")
    print(f"  -> 最終 shape: {df.shape}")

    print("\n✅ 清理完成。建議再簡單看看：df.head(3), df.info(), df.describe()")
    return df

In [8]:
df_clean = clean_data_impute(df)

===== 原始資料摘要 =====
shape: (20718, 28)
 缺失值統計（>0 的欄位）：
Description         876
Stream              576
Comments            569
Likes               541
Url_youtube         470
Licensed            470
official_video      470
Views               470
Channel             470
Title               470
Danceability          2
Key                   2
Energy                2
Speechiness           2
Loudness              2
Tempo                 2
Duration_ms           2
Instrumentalness      2
Acousticness          2
Liveness              2
  ... 另有 1 個欄位省略

===== 刪除冗餘欄位 =====
  將移除欄位：['Unnamed: 0']
  -> shape: (20718, 27)

=====型別標準化（數值欄轉 numeric）=====
  嘗試轉為數值的欄位：['Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'Views', 'Likes', 'Comments', 'Stream']
  轉型造成的 NaN 變化（正值=新增 NaN，代表原本是非數字字串）：
Series([], )
  -> shape: (20718, 27)

===== 布林欄位正規化 =====
  嘗試正規化布林欄位：['Licensed', 'official_video']
  Licensed 值分佈：


In [9]:
df_clean.to_csv(r"C:\Users\hu041\OneDrive\桌面\nschool\Spotify_Youtube_clean.csv", index=False, encoding="utf-8-sig")